# Analiza podatkov KRsKolesom

In [100]:
import json
from urllib.request import urlopen
import pandas as pd
import datetime

In [101]:
station_data_url = "http://www.krskolesom.si/api/locationData"
station_info_fn = "krskolesom_station_info.csv"
station_data_fn = (
    "krskolesom_station_data_" + datetime.datetime.now().strftime("%y%m") + ".csv"
)
bicikelj_log = (
    "krskolesom_station_data_" + datetime.datetime.now().strftime("%y%m") + ".log"
)
# Open log file, all messages are written to log
log_file = open(bicikelj_log, "a")
print(
    "KRsKolesom",
    datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    sep=",",
    end=",",
    file=log_file,
)

In [102]:
response = urlopen(station_data_url)
if response.code == 200:
    data = response.read().decode("utf-8")
else:
    print("Wrong response from ", station_data_url)

In [103]:
station_data_json = json.loads(data)

In [104]:
station_data = (
    pd.DataFrame(station_data_json).sort_values(["id"]).reset_index(drop=True)
)

In [105]:
station_data.columns

Index(['id', 'latitude', 'longitude', 'lowOnBikes', 'lowOnLocks',
       'mapViewRangeInMeters', 'name', 'numberOfBikes', 'numberOfFreeBikes',
       'numberOfFreeLocks', 'numberOfLocks', 'numberOfTotalFaulty', 'street'],
      dtype='object')

In [106]:
station_info = station_data[
    ["id", "latitude", "longitude", "mapViewRangeInMeters", "name", "street"]
].copy()
station_info

,id,latitude,longitude,mapViewRangeInMeters,name,street
0,14,46.242385,14.356695,500,Mestna knjižnica Kranj,Slovenski trg 3 3/4
1,15,46.238504,14.358165,500,Čebelica,Cesta 1. maja 1a 3/4
2,16,46.249861,14.364549,500,Športni center,Partizanska cesta 35 3/4
3,17,46.237517,14.364640,500,Planina otok,Cesta talcev 62 3/4
4,18,46.252042,14.357768,500,Šorlijevo naselje,Šorlijeva ulica 14 3/4
5,19,46.245765,14.354898,500,Osrednja lekarna,Bleiweisova cesta 8 3/4
6,24,46.248277,14.349582,500,Zlato Polje,Kidričeva cesta 2 3/4
7,25,46.249516,14.352967,500,Zdravstveni dom,Gosposvetska 19 3/4
8,26,46.248533,14.359710,500,Vodovodni Stolp,Oldhamska cesta 5 3/4
9,27,46.240697,14.369023,500,Planina III,Ulica Janka Puclja 30 3/4


In [107]:
# Station info to csv
station_info.to_csv(station_info_fn, index=False)

In [108]:
station_data_now = station_data[
    [
        "id",
        "lowOnBikes",
        "lowOnLocks",
        "numberOfBikes",
        "numberOfFreeBikes",
        "numberOfFreeLocks",
        "numberOfLocks",
        "numberOfTotalFaulty",
    ]
].copy()
station_data_now

,id,lowOnBikes,lowOnLocks,numberOfBikes,numberOfFreeBikes,numberOfFreeLocks,numberOfLocks,numberOfTotalFaulty
0,14,False,False,2,2,8,10,0
1,15,False,False,4,4,6,10,0
2,16,False,False,2,2,8,10,0
3,17,False,False,5,5,5,10,0
4,18,False,False,4,4,6,10,0
5,19,False,False,1,1,9,10,0
6,24,False,False,6,6,4,10,0
7,25,True,False,0,0,2,2,0
8,26,False,False,6,6,4,10,0
9,27,False,False,7,7,3,10,0


In [109]:
station_data_now["datetime"] = datetime.datetime.now()
station_data_now

,id,lowOnBikes,lowOnLocks,numberOfBikes,numberOfFreeBikes,numberOfFreeLocks,numberOfLocks,numberOfTotalFaulty,datetime
0,14,False,False,2,2,8,10,0,2017-11-12 16:07:53.003986
1,15,False,False,4,4,6,10,0,2017-11-12 16:07:53.003986
2,16,False,False,2,2,8,10,0,2017-11-12 16:07:53.003986
3,17,False,False,5,5,5,10,0,2017-11-12 16:07:53.003986
4,18,False,False,4,4,6,10,0,2017-11-12 16:07:53.003986
5,19,False,False,1,1,9,10,0,2017-11-12 16:07:53.003986
6,24,False,False,6,6,4,10,0,2017-11-12 16:07:53.003986
7,25,True,False,0,0,2,2,0,2017-11-12 16:07:53.003986
8,26,False,False,6,6,4,10,0,2017-11-12 16:07:53.003986
9,27,False,False,7,7,3,10,0,2017-11-12 16:07:53.003986


In [110]:
# Add to file
try:
    station_data_full = pd.read_csv(station_data_fn, parse_dates=["datetime"])
    len_before = len(station_data_full.index)
    station_data_full = station_data_full.append(station_data_now).drop_duplicates()
    len_after = len(station_data_full.index)
    station_data_full.to_csv(station_data_fn, index=False)
    print(len_after - len_before, "Added", sep=",", file=log_file)
except:
    station_data_now.to_csv(station_data_fn, index=False)
    len_after = len(station_data_now.index)
    print(len_after, "Added", sep=",", file=log_file)

In [111]:
# Station info to csv
station_data_now.to_csv(station_data_fn, index=False, mode="a", header=False)

In [112]:
log_file.close()